# ML version of quantum state preparation

In [1]:
# imports

import numpy as np
import matplotlib.pyplot as plt

from src import visuals as vis
%matplotlib qt

# definitions

rho_x = np.array([[0,1],[1,0]])
rho_y = np.array([[0,-1j],[1j,0]])
rho_z = np.array([[1,0],[0,-1]])

rho_m = np.array([[0,1],[0,0]])
rho_p = np.array([[0,0],[1,0]])

psi_0 = np.array([1,0])
psi_1 = np.array([0,1])

def H(w, g):
    return w * rho_z + g * rho_x


def lindbladian(rho, H, L, gamma, dt):
    """hbar = 1"""
    Ldag = L.conj().T
    drho = -1j * (H @ rho - rho @ H) + gamma * (L @ rho @ Ldag - 0.5 * ((Ldag @ L) @ rho + rho @ (Ldag @ L) ))
    rho_new = rho + drho*dt
    return rho_new

rho = np.outer(np.conj(psi_0), psi_0)



# simulation

N = 100
T = 1

times = np.linspace(0,T,N)
s_x = np.zeros(N, dtype='complex')
s_y = np.zeros(N, dtype='complex')
s_z = np.zeros(N, dtype='complex')
tr = np.zeros(N)
pur = np.zeros(N)
fid = np.zeros(N)

rho_init = np.outer(np.conj(psi_0), psi_0)

w = 5
g = 4
L = rho_p
gamma = 1
dt = T/N

rho = rho_init
for i in range(N):
    s_x[i] = np.trace(rho @ rho_x)
    s_y[i] = np.trace(rho @ rho_y)
    s_z[i] = np.trace(rho @ rho_z)
    
    tr[i] = np.trace(rho)
    pur[i] = np.trace(rho @ rho)
    fid[i] = np.trace(rho @ rho_init)
    
    rho = lindbladian(rho, H(w,g), L, gamma, dt)

vis.plotBlochSphereTrajectory(s_x, s_y, s_z, 100)

C:\Users\Michal\AppData\Local\Temp\ipykernel_25496\2359911832.py:63: ComplexWarning: Casting complex values to real discards the imaginary part
  tr[i] = np.trace(rho)
C:\Users\Michal\AppData\Local\Temp\ipykernel_25496\2359911832.py:64: ComplexWarning: Casting complex values to real discards the imaginary part
  pur[i] = np.trace(rho @ rho)
C:\Users\Michal\AppData\Local\Temp\ipykernel_25496\2359911832.py:65: ComplexWarning: Casting complex values to real discards the imaginary part
  fid[i] = np.trace(rho @ rho_init)
c:\Users\Michal\miniconda3\envs\quantum\lib\site-packages\matplotlib\cbook.py:1762: ComplexWarning: Casting complex values to real discards the imaginary part
  return math.isfinite(val)
c:\Users\Michal\miniconda3\envs\quantum\lib\site-packages\matplotlib\cbook.py:1398: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

plotting_spins = 1
if plotting_spins:
    ax.plot(times, s_x.real, c='C1', label='x')
    #ax.plot(s_x.imag, c='C1', ls='-.')
    ax.plot(times, s_y.real, c='C2', label='y')
    #ax.plot(s_y.imag, c='C2', ls='-.')
    ax.plot(times, s_z.real, c='C3', label='z')
    #ax.plot(s_z.imag, c='C3', ls='-.')
    ax.hlines([0], [0],[times[-1]], colors=['black'], linestyles=[':'])

plotting_markers = not plotting_spins
if plotting_markers:
    ax.plot(times, tr, ls='--', c='black', label='Trace')
    ax.plot(times, pur, ls='--', c='pink', label='Purity')
    ax.plot(times, fid, ls='--', c='cyan', label='Fidelity')

ax.legend()
fig.show()


c:\Users\Michal\miniconda3\envs\quantum\lib\site-packages\matplotlib\cbook.py:1398: ComplexWarning: Casting complex values to real discards the imaginary part
  return np.asarray(x, float)


In [2]:

import pylab as plt
import numpy as np

# Make some data
n=1000
x=np.linspace(0,100,n)
y=np.sin(x)

# Your coloring array
T=np.linspace(0,1,np.size(x))**2
fig = plt.figure()
ax = fig.add_subplot(111)

# Segment plot and color depending on T
s = 10 # Segment length
for i in range(0,n-s,s):
    ax.plot(x[i:i+s+1],y[i:i+s+1],color=(0.0,0.5,T[i]))